In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.5,
    base_url="http://localhost:11434",
    stream_usage=True  # 流模式需开启统计
)

在构建聊天机器人时，将对话状态传入和传出链至关重要。该类RunnableWithMessageHistory允许我们将消息历史记录添加到某些类型的链中。它包装另一个 Runnable 并为其管理聊天消息历史记录。具体来说，它在将对话传递给 Runnable 之前加载对话中的先前消息，并在调用 Runnable 之后将生成的响应保存为消息。该类还通过使用 a 保存每个对话来启用多个对话session_id- 然后它期望session_id在调用 Runnable 时在配置中传递 a，并使用它来查找相关的对话历史记录。

from langchain_core.runnables.history import RunnableWithMessageHistory


with_message_history = RunnableWithMessageHistory(
    # The underlying runnable
    runnable,  
    # A function that takes in a session id and returns a memory object
    get_session_history,  
    # Other parameters that may be needed to align the inputs/outputs
    # of the Runnable with the memory object
    ...  
)

with_message_history.invoke(
    # The same input as before
    {"ability": "math", "input": "What does cosine mean?"},
    # Configuration specifying the `session_id`,
    # which controls which conversation to load
    config={"configurable": {"session_id": "abc123"}},
)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

In [ ]:

from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history,
)


In [ ]:
runnable_with_history.invoke(
    [HumanMessage(content="hi - im bob!")],
    config={"configurable": {"session_id": "1"}},
)

In [ ]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1"}},
)

In [ ]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1a"}},
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
# You're an assistant who speaks in English. Respond in 20 words or fewer.
# 历史记录：Hi, how are you?
# 输入：What's the weather like today?

runnable = prompt | llm

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)